In [33]:
import pandas as pd
import os

def load_data(file_name, data_type = None):
    """
    Reads a CSV file from the 'data' directory into a pandas DataFrame.

    Args:
        file_name (str): The name of the CSV file to load.

    Returns:
        pandas.DataFrame: The loaded data.
    """
    # Construct the relative path from the notebook to the data folder
    # Notebook is in 'tests/notebooks/', so we go up two levels
    base_path = os.path.abspath(os.path.join(os.path.dirname(file_name), '..', '..'))
    if data_type == 'synthetic_train':
        data_path = os.path.join(base_path, 'data/input_data/synth_data/fraudTrain/', file_name)
    elif data_type == 'synthetic_test':
        data_path = os.path.join(base_path, 'data/input_data/synth_data/fraudTest/', file_name)
    elif data_type == 'ieee_fraud_detection':
        data_path = os.path.join(base_path, 'data/input_data/synth_data/fraudTrain/', file_name)
    else:
        raise ValueError('data_type must be either synthetic_train, synthetic_test or ieee_fraud_detection')

    if not os.path.exists(data_path):
        print(f"Error: File not found at {data_path}")
        return None

    return pd.read_csv(data_path)


In [35]:
synt_train_data = load_data('fraudTrain.zip', data_type = 'synthetic_train')

In [36]:
synt_train_data.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [38]:
synt_test_data = load_data('fraudTest.zip', data_type = 'synthetic_test')

In [39]:
synt_test_data.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05,5.557190e+05,555719.000000,555719.000000,555719.000000
mean,277859.000000,4.178387e+17,69.392810,48842.628015,38.543253,-90.231325,8.822189e+04,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,1.309837e+18,156.745941,26855.283328,5.061336,13.721780,3.003909e+05,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,1.800429e+14,9.630000,26292.000000,34.668900,-96.798000,7.410000e+02,1.376029e+09,34.755302,-96.905129,0.000000
50%,277859.000000,3.521417e+15,47.290000,48174.000000,39.371600,-87.476900,2.408000e+03,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,4.635331e+15,83.010000,72011.000000,41.894800,-80.175200,1.968500e+04,1.385867e+09,41.954163,-80.264637,0.000000
max,555718.000000,4.992346e+18,22768.110000,99921.000000,65.689900,-67.950300,2.906700e+06,1.388534e+09,66.679297,-66.952026,1.000000


In [41]:
ieee_data = load_data('ieee_fraud_detection.zip/test_identity.csv', data_type = 'ieee_fraud_detection')


Error: File not found at C:\Users\Marco\PycharmProjects\fraud_detection_adversarial\tests\data/input_data/synth_data/fraudTrain/ieee_fraud_detection.zip/test_identity.csv
